In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import random

In [10]:
bs, bptt = 64, 70

In [11]:
#python -m spacy download fr
my_tok = spacy.load('fr')

def my_spacy_tok(x):
    return [tok.text for tok in my_tok.tokenizer(x)]

In [12]:
PATH = 'data/txt/wordpress'
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
md = LanguageModelData.from_text_files(PATH, TEXT, train='.', validation='.', test='.',
                                       bs=bs, bptt=bptt, min_freq=10)

In [13]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(123, 4002, 1, 558973)

In [14]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [15]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [16]:
warnings.filterwarnings('ignore',category=UserWarning)

In [17]:
learner.fit(3e-3, 1, wds=1e-6)

epoch      trn_loss   val_loss                               
    0      5.446745   5.175616  



[5.175615642837401]

In [22]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)], device=-1)

In [23]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0).unsqueeze(0))

    m[0].bs=bs

In [24]:
m = learner.model.cpu()

In [37]:
sample_model(m, "j'aime", l=100)

...de la ville de la ville , je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de la ville de la ville . je n’ est pas de 